<img src="figure/clairvoyance_logo.png">

# Clairvoyance: Transfer learning on time-series prediction

## ML-AIM (http://vanderschaar-lab.com/)

This notebook describes the user-guide of a transfer learning on time-series predictions application using Clairvoyance framework. The separate tutorial for the time-series prediction can be found in tutorial_prediction.ipynb. Transfer learning is defined as following: transfering the knowledge of the trained model (trained on different dataset) to improve the predictive model to the interested dataset. For instance, transfer the knowledge of the trained time-series prediction model on Hospital A to improve the predictive model for Hospital B.
 
<img src="figure/transfer-learning-definition.png">

To run this tutorial, you need:
### Temporal and static datasets for training and testing

If users come with their own temporal and static datasets for training and testing, the users should save those files as 'data_name_temporal_train_data_eav.csv.gz', 'data_name_static_train_data.csv.gz', 'data_name_temporal_test_data_eav.csv.gz', 'data_name_static_test_data.csv.gz' in '../datasets/data/data_name/' directory.


## Transfer learning on time-series prediction pipeline summary

<img src="figure/transfer-learning-block-diagram.png">

### Step 1: Load dataset
  - Extract csv files from the original raw datasets in ../datasets/data/data_name/ directory.  
  
### Step 2: Preprocess dataset
  - Preprocessing the raw data using various filters such as (1) replacing negative values to NaN, (2) do one-hot encidng for certain features, (3) do normalization.  
  
### Step 3: Define problem
  - Set the time-series prediction problem that we want to solve. Set the problem (whether it is one-shot or online prediction), set the label, set the maximum sequence length, and set the treatment features. We also define the metric for evaluation and the task itself (whether classification or regression).

### Step 4: Impute dataset
  - Impute missing values in the preprocessed static and temporal datasets and return complete datasets.
  
### Step 5: Feature selection
  - Select the relevant static and temporal features to the labels. You can skip the feature selection (set feature selection method = None).
  
### Step 6: Original time-series model fit and predict
  - After finishing the data preparation, we define the original predictive models and train the model using the training dataset. After training, we use the trained model to predict the labels of the testing dataset.
  
### Step 7: Transfer learning time-series model fit and predict
  - After finishing the data preparation and original predictive model training, we define the transfer learning model and train the model on top of trained original predictive model using the transfer dataset. After training, we use the trained transfer model to predict the labels of the testing dataset.

### Step 8: Visualize results
  - Visualize the various results such as performance and predictions.

## Step 0: Import necessary packages

Import necessary packages for the entire tutorials.

In [1]:
# Necessary packages
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import warnings
warnings.filterwarnings('ignore')
import sys
sys.path.append('../')

from utils import PipelineComposer

## Step 1: Load dataset

Extract temporal and static datasets from 'data_name_temporal_train_data_eav.csv.gz', 'data_name_static_train_data.csv.gz', 'data_name_temporal_test_data_eav.csv.gz', 'data_name_static_test_data.csv.gz' in '../datasets/data/data_name/' directory.

- CSVLoader: Load csv files from the original raw datasets in ../datasets/data/data_name/ directory.
- file_names: mimic in this tutorial.

In [2]:
from datasets import CSVLoader

# Define data name
data_name = 'mimic'
# Define data dictionary
data_directory = '../datasets/data/'+data_name + '/' + data_name + '_'

# Load train and test datasets
data_loader_training = CSVLoader(static_file=data_directory + 'static_train_data.csv.gz',
                                 temporal_file=data_directory + 'temporal_train_data_eav.csv.gz')

data_loader_testing = CSVLoader(static_file=data_directory + 'static_test_data.csv.gz',
                                temporal_file=data_directory + 'temporal_test_data_eav.csv.gz')

dataset_training = data_loader_training.load()
dataset_testing = data_loader_testing.load()

print('Finish data loading.')

Finish data loading.


## Step 2: Preprocess dataset

Preprocess the raw data using multiple filters. In this tutorial, we replace all the negative values to NaN (using NegativeFilter), do one-hot encoding on 'admission_type' feature (using OneHotEncoder), and do MinMax Normalization (using Normalization). Preprocessing is done for both training and testing datasets. 
  - NegativeFilter: Replace negative values to NaN
  - OneHotEncoder: One hot encoding certain features
    - one_hot_encoding: input features that need to be one-hot encoded
  - Normalization (3 options): MinMax, Standard, None

In [3]:
from preprocessing import FilterNegative, OneHotEncoder, Normalizer

# (1) filter out negative values
negative_filter = FilterNegative()
# (2) one-hot encode categorical features
one_hot_encoding = 'admission_type'
onehot_encoder = OneHotEncoder(one_hot_encoding_features=[one_hot_encoding])
# (3) Normalize features: 3 options (minmax, standard, none)
normalization = 'minmax'
normalizer = Normalizer(normalization)

# Data preprocessing
filter_pipeline = PipelineComposer(negative_filter, onehot_encoder, normalizer)

dataset_training = filter_pipeline.fit_transform(dataset_training)
dataset_testing = filter_pipeline.transform(dataset_testing)

print('Finish preprocessing.')

Finish preprocessing.


## Step 3: Define problem   

Set the time-series prediction problem that we want to solve. Set the problem (whether it is one-shot or online prediction), set the label, set the maximum sequence length, and set the treatment features. We also define the metric for evaluation and the task itself (whether classification or regression). In this tutorial, we predict whether the patients will die at the end of the hospital stay (one-shot setting).
  - problem: 'one-shot'(one time prediction) or 'online'(rolling window prediction)
    - 'one-shot': one time prediction at the end of the time-series 
    - 'online': preditcion at every time stamps of the time-series
  - max_seq_len: maximum sequence length of time-series sequence
  - label_name: the column name for the label(s)
  - treatment: the column name for treatments
  - window: x-hour ahead prediction (for online prediction).
  
  - other parameters:
    - metric_name: auc, apr, mse, mae
    - task: classification or regression

In [4]:
from preprocessing import ProblemMaker

# Define parameters
problem = 'one-shot'
max_seq_len = 24
label_name = 'death'
treatment = None
window = 0

# Define problem 
problem_maker = ProblemMaker(problem=problem, label=[label_name],
                             max_seq_len=max_seq_len, treatment=treatment, window = window)

dataset_training = problem_maker.fit_transform(dataset_training)
dataset_testing = problem_maker.fit_transform(dataset_testing)

# Set other parameters
metric_name = 'auc'
task = 'classification'

metric_sets = [metric_name]
metric_parameters =  {'problem': problem, 'label_name': [label_name]}

print('Finish defining problem.')

100%|██████████| 4610/4610 [00:08<00:00, 531.60it/s]


Finish defining problem.


## Step 4: Impute dataset

Impute missing values in the preprocessed static and temporal datasets and return complete datasets.
  - Static imputation (6 options): mean, median, mice, missforest, knn, gain
  - Temporal imputation (8 options): mean, median, linear, quadratic, cubic, spline, mrnn, tgain

In [5]:
from imputation import Imputation

# Set imputation models
static_imputation_model = 'median'
temporal_imputation_model = 'median'

# Impute the missing data
static_imputation = Imputation(imputation_model_name = static_imputation_model, data_type = 'static')
temporal_imputation = Imputation(imputation_model_name = temporal_imputation_model, data_type = 'temporal')

imputation_pipeline = PipelineComposer(static_imputation, temporal_imputation)

dataset_training = imputation_pipeline.fit_transform(dataset_training)
dataset_testing = imputation_pipeline.transform(dataset_testing)

print('Finish imputation.')

Using TensorFlow backend.


Finish imputation.


## Step 5: Feature selection

Select the relevant static and temporal features to the labels. If you do not want, we can skip the feature selection (set feature selection method = None).
  - feature selection method: 
      - feature_selection_model: greedy-addition, greedy-deletion, recursive-addition, recursive-deletion, None
      - feature_number: selected featuer number

In [6]:
from feature_selection import FeatureSelection

# Set feature selection parameters
static_feature_selection_model = None
temporal_feature_selection_model = None
static_feature_selection_number = None
temporal_feature_selection_number = None

# Select relevant features
static_feature_selection = \
FeatureSelection(feature_selection_model_name = static_feature_selection_model,
                 feature_type = 'static', feature_number = static_feature_selection_number,
                 task = task, metric_name = metric_name,
                 metric_parameters = metric_parameters)

temporal_feature_selection = \
FeatureSelection(feature_selection_model_name = temporal_feature_selection_model,
                 feature_type = 'temporal', feature_number = temporal_feature_selection_number,
                 task = task, metric_name = metric_name,
                 metric_parameters = metric_parameters)

feature_selection_pipeline = PipelineComposer(static_feature_selection, temporal_feature_selection)

dataset_training = feature_selection_pipeline.fit_transform(dataset_training)
dataset_testing = feature_selection_pipeline.transform(dataset_testing)

print('Finish feature selection.')

Finish feature selection.


## Step 6: Original time-series model fit and predict

After finishing the data preparation, we define the original predictive models (6 options, RNN, GRU, LSTM, Attention, Temporal CNN, and Transformer), and train the original model using the training dataset. We set validation set as the 20% of the training set for early stopping and best model saving. After training, we use the original trained model to predict the labels of the testing dataset.

- predictor_parameters:
  - ori_model_name: rnn, gru, lstm
  - ori_model_parameters: network parameters such as numer of layers
    - h_dim: hidden dimensions
    - n_layer: layer number
    - n_head: head number (only for transformer model)
    - batch_size: number of samples in mini-batch
    - epochs: number of epochs
    - learning_rate: learning rate
  - static_mode: how to utilize static features (concatenate or None)
  - time_mode: how to utilize time information (concatenate or None)

In [7]:
from prediction import prediction

# Set original predictive model
ori_model_name = 'gru'

# Set original model parameters
ori_model_parameters = {'h_dim': 100,
                        'n_layer': 2,
                        'n_head': 2,
                        'batch_size': 128,
                        'epoch': 20,
                        'model_type': ori_model_name,
                        'learning_rate': 0.001,
                        'static_mode': 'concatenate',
                        'time_mode': 'concatenate',
                        'verbose': True}

# Set up validation for early stopping and best model saving
dataset_training.train_val_test_split(prob_val=0.2, prob_test = 0.0)

# Train the original predictive model
ori_pred_class = prediction(ori_model_name, ori_model_parameters, task)
ori_pred_class.fit(dataset_training)
# Return the predictions on the testing set by the original predictive model
ori_test_y_hat = ori_pred_class.predict(dataset_testing)

print('Finish predictor model training and testing.')

Train on 14792 samples, validate on 3698 samples
Epoch 1/20
14792/14792 [==============================] - 5s 343us/sample - loss: 0.3233 - val_loss: 0.3153
Epoch 2/20
14792/14792 [==============================] - 4s 269us/sample - loss: 0.2931 - val_loss: 0.3022
Epoch 3/20
14792/14792 [==============================] - 4s 276us/sample - loss: 0.2825 - val_loss: 0.2986
Epoch 4/20
14792/14792 [==============================] - 4s 274us/sample - loss: 0.2766 - val_loss: 0.2901
Epoch 5/20
14792/14792 [==============================] - 4s 268us/sample - loss: 0.2740 - val_loss: 0.2836
Epoch 6/20
14792/14792 [==============================] - 4s 270us/sample - loss: 0.2737 - val_loss: 0.2953
Epoch 7/20
14792/14792 [==============================] - 4s 271us/sample - loss: 0.2673 - val_loss: 0.2789
Epoch 8/20
14792/14792 [==============================] - 4s 275us/sample - loss: 0.2651 - val_loss: 0.2860
Epoch 9/20
14792/14792 [==============================] - 4s 272us/sample - loss: 0.260

## Step 7: Transfer learning time-series model fit and predict

After finishing the data preparation and original predictive model training, we define the transfer learning predictive models (3 options, RNN, GRU, LSTM), and train the model using the transfer dataset. After training, we use the trained transfer model to predict the labels of the testing dataset.

- predictor_parameters:
  - trans_model_name: rnn, gru, lstm
  - trans_model_parameters: network parameters such as numer of layers
    - h_dim: hidden dimensions
    - n_layer: layer number
    - n_head: head number (only for transformer model)
    - batch_size: number of samples in mini-batch
    - epochs: number of epochs
    - learning_rate: learning rate
  - static_mode: how to utilize static features (concatenate or None)
  - time_mode: how to utilize time information (concatenate or None)

In [11]:
from prediction import TransferLearning

# Set transfer learning model
trans_model_name = 'gru'

# Set model parameters
trans_model_parameters = {'h_dim': 100,
                        'n_layer': 2,
                        'n_head': 2,
                        'batch_size': 128,
                        'epoch': 50,
                        'model_type': trans_model_name,
                        'learning_rate': 0.001,
                        'static_mode': 'concatenate',
                        'time_mode': 'concatenate',
                        'verbose': True}

trans_class = TransferLearning(ori_pred_class, trans_model_parameters, task)
trans_class.fit(dataset_testing)
trans_test_y_hat = trans_class.predict(dataset_testing) 
    
print('Finish transfer learning model training and testing.') 

Train on 4610 samples, validate on 4610 samples
Epoch 1/500
4610/4610 [==============================] - 0s 107us/sample - loss: 0.3398 - val_loss: 0.2662
Epoch 2/500
4610/4610 [==============================] - 0s 12us/sample - loss: 0.2573 - val_loss: 0.2502
Epoch 3/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2471 - val_loss: 0.2443
Epoch 4/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2434 - val_loss: 0.2417
Epoch 5/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2412 - val_loss: 0.2406
Epoch 6/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2410 - val_loss: 0.2403
Epoch 7/500
4610/4610 [==============================] - 0s 10us/sample - loss: 0.2409 - val_loss: 0.2403
Epoch 8/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2427 - val_loss: 0.2401
Epoch 9/500
4610/4610 [==============================] - 0s 11us/sample - loss: 0.2403 - val_loss: 0.24

## Step 8: Visualize results

(1) Visualize the performance of the trained model.

In [12]:
from evaluation import Metrics
from evaluation import print_performance

# Evaluate predictor model
_, _, test_y, _, _ = dataset_testing.get_fold(0, 'test')
ori_result = Metrics(metric_sets, metric_parameters).evaluate(test_y, ori_test_y_hat)
trans_result = Metrics(metric_sets, metric_parameters).evaluate(test_y, trans_test_y_hat)
print('Finish original and transfer predictor model evaluation.')

print('Overall performance of the original model')
print_performance(ori_result, metric_sets, metric_parameters)
print('Overall performance of the transfer model')
print_performance(trans_result, metric_sets, metric_parameters)

Finish original and transfer predictor model evaluation.
Overall performance of the original model
+--------------+-------+
| metric/label | death |
+==============+=======+
|     auc      | 0.865 |
+--------------+-------+
Overall performance of the transfer model
+--------------+-------+
| metric/label | death |
+==============+=======+
|     auc      | 0.880 |
+--------------+-------+


## Step 8: Visualize results

(2) Visualize the predictions of a certain patient by trained predictive model.

In [13]:
from evaluation import print_prediction

# Set the patient index for visualization
index = [1]

print('Each prediction of the original model')
print_prediction(ori_test_y_hat[index], metric_parameters)
print('Each prediction of the transfer model')
print_prediction(trans_test_y_hat[index], metric_parameters)

Each prediction of the original model
+----------+-------+
| id/label | death |
+==========+=======+
|    1     | 0.056 |
+----------+-------+
Each prediction of the transfer model
+----------+-------+
| id/label | death |
+==========+=======+
|    1     | 0.048 |
+----------+-------+
